<a href="https://colab.research.google.com/github/matheussato/Analise-Cancer-Mama/blob/main/EstudoCancerDeMamaCP1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports

In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
df = pd.read_csv("breast-cancer.data")

### Renomeando as colunas para melhor entendimento do dataset

In [ ]:
df.rename(columns={'no-recurrence-events' : 'eventos_recorrentes'}, inplace=True)
df.rename(columns={'30-39': 'idade'}, inplace=True)
df.rename(columns={'premeno': 'menopausa'}, inplace=True)
df.rename(columns={'30-34': 'tamanho_tumor'}, inplace=True)
df.rename(columns={'0-2': 'numero_linfonodos'}, inplace=True)
df.rename(columns={'no': 'pen_linfonodos'}, inplace=True)
df.rename(columns={'3': 'grau_malignidade'}, inplace=True)
df.rename(columns={'left': 'mama'}, inplace=True)
df.rename(columns={'left_low': 'quadrante_afetado'}, inplace=True)
df.rename(columns={'no.1': 'radioterapia'}, inplace=True)
df.head(10)

,eventos_recorrentes,idade,menopausa,tamanho_tumor,numero_linfonodos,pen_linfonodos,grau_malignidade,mama,quadrante_afetado,radioterapia
0,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,right_up,no
1,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,left,left_low,no
2,no-recurrence-events,60-69,ge40,15-19,0-2,no,2,right,left_up,no
3,no-recurrence-events,40-49,premeno,0-4,0-2,no,2,right,right_low,no
4,no-recurrence-events,60-69,ge40,15-19,0-2,no,2,left,left_low,no
5,no-recurrence-events,50-59,premeno,25-29,0-2,no,2,left,left_low,no
6,no-recurrence-events,60-69,ge40,20-24,0-2,no,1,left,left_low,no
7,no-recurrence-events,40-49,premeno,50-54,0-2,no,2,left,left_low,no
8,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,left_up,no
9,no-recurrence-events,40-49,premeno,0-4,0-2,no,3,left,central,no


### Colocando o dataframe em ordem por eventos_recorrentes

In [ ]:
df = df.sort_values('eventos_recorrentes', ascending=False)
df

,eventos_recorrentes,idade,menopausa,tamanho_tumor,numero_linfonodos,pen_linfonodos,grau_malignidade,mama,quadrante_afetado,radioterapia
284,recurrence-events,50-59,ge40,30-34,3-5,no,3,left,left_low,no
231,recurrence-events,30-39,premeno,30-34,3-5,no,3,right,left_up,yes
223,recurrence-events,30-39,premeno,25-29,3-5,yes,3,left,left_low,yes
224,recurrence-events,40-49,ge40,20-24,3-5,no,3,right,left_low,yes
225,recurrence-events,40-49,premeno,30-34,15-17,yes,3,left,left_low,no
...,...,...,...,...,...,...,...,...,...,...
101,no-recurrence-events,50-59,premeno,25-29,0-2,no,2,right,right_low,no
102,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,left,right_low,no
103,no-recurrence-events,40-49,premeno,10-14,0-2,no,2,right,left_low,no
104,no-recurrence-events,40-49,premeno,30-34,0-2,no,1,right,left_up,no


### Deletando coluna que não utilizaremos

In [ ]:
df = df.drop('mama', axis=1)

Pesquisando mais sobre o assunto entendemos que o lado afetado não interfere diretamente nos casos de recorrência da enfermidade,

### Calculando o percentual de dados faltantes do dataset

In [ ]:
def calcula_percentual_indefinido(dataframe, coluna):
  indefinidos = dataframe.loc[pd.isna(dataframe[coluna]),coluna].shape[0]
  total = dataframe[coluna].shape[0]

  return (indefinidos/total)*100

In [ ]:
calcula_percentual_indefinido(df, "idade")

0.0

In [ ]:
calcula_percentual_indefinido(df, "menopausa")

0.0

In [ ]:
calcula_percentual_indefinido(df, "tamanho_tumor")

0.0

In [ ]:
calcula_percentual_indefinido(df, "numero_linfonodos")

0.0

In [ ]:
calcula_percentual_indefinido(df, "pen_linfonodos")

0.0

In [ ]:
calcula_percentual_indefinido(df, "grau_malignidade")

0.0

In [ ]:
calcula_percentual_indefinido(df, "radioterapia")

0.0

In [ ]:
calcula_percentual_indefinido(df, "quadrante_afetado")

0.0

In [ ]:
calcula_percentual_indefinido(df, "radioterapia")

0.0

### Tirar '?' que achamos no meio das colunas

In [ ]:
interrogacao = []
for coluna in df:
  quantidade_interrog = len(df.loc[df[coluna]=='?'])
  interrogacao.append(quantidade_interrog)


In [ ]:
df.loc[df['pen_linfonodos']=='?', 'pen_linfonodos'] = np.nan
df.loc[df['quadrante_afetado']=='?', 'quadrante_afetado'] = np.nan

In [ ]:
df = df[~pd.isna(df.quadrante_afetado)].reset_index()
df = df.drop(columns='index')

### Verificar se os valores são categóricos

In [ ]:
df['idade'].unique()

array(['50-59', '30-39', '40-49', '60-69', '70-79', '20-29'], dtype=object)

In [ ]:
df['menopausa'].unique()

array(['ge40', 'premeno', 'lt40'], dtype=object)

In [ ]:
df['tamanho_tumor'].unique()

array(['30-34', '25-29', '20-24', '40-44', '45-49', '50-54', '15-19',
       '35-39', '0-4', '10-14', '5-9'], dtype=object)

In [ ]:
df['numero_linfonodos'].unique()

array(['3-5', '15-17', '0-2', '9-11', '6-8', '12-14', '24-26'],
      dtype=object)

In [ ]:
df['pen_linfonodos'].unique()

array(['no', 'yes', nan], dtype=object)

In [ ]:
df['grau_malignidade'].unique()

array([3, 1, 2])

In [ ]:
df['quadrante_afetado'].unique()

array(['left_low', 'left_up', 'right_up', 'right_low', 'central'],
      dtype=object)

In [ ]:
df['radioterapia'].unique()

array(['no', 'yes'], dtype=object)

Transformando os valores das colunas em vetores exemplo = [0,1],[1,0] etc

In [ ]:
def one_hot(categorias, valor):
  categorias = list(set(categorias))
  indice = categorias.index(valor)
  vetor = np.zeros(len(categorias))
  vetor[indice] = 1
  return vetor

In [ ]:
df.head(10)

In [ ]:
df.eventos_recorrentes = [one_hot(df.eventos_recorrentes, _) for _ in df.eventos_recorrentes]
df.idade = [one_hot(df.idade, _) for _ in df.idade]
df.menopausa = [one_hot(df.menopausa, _) for _ in df.menopausa]
df.tamanho_tumor = [one_hot(df.tamanho_tumor, _) for _ in df.tamanho_tumor]
df.numero_linfonodos = [one_hot(df.numero_linfonodos, _) for _ in df.numero_linfonodos]
df.pen_linfonodos = [one_hot(df.pen_linfonodos, _) for _ in df.pen_linfonodos]
df.grau_malignidade = [one_hot(df.grau_malignidade, _) for _ in df.grau_malignidade]
df.quadrante_afetado = [one_hot(df.quadrante_afetado, _) for _ in df.quadrante_afetado]
df.radioterapia = [one_hot(df.radioterapia, _) for _ in df.radioterapia]

In [ ]:
df

In [ ]:
labels = df.eventos_recorrentes
df = df.drop(columns = 'eventos_recorrentes')


In [ ]:
def vetorizar(idade, menopausa, tamanho_tumor, numero_linfonodos, pen_linfonodos, grau_malignidade, quadrante_afetado, radioterapia):
    return idade.tolist() + menopausa.tolist() + tamanho_tumor.tolist() + numero_linfonodos.tolist() \
    + pen_linfonodos.tolist() + grau_malignidade.tolist() + quadrante_afetado.tolist() + radioterapia.tolist()

In [ ]:
df

In [ ]:
entrada = [vetorizar(*_) for _ in df.values]

In [ ]:
entrada

In [ ]:
np.savetxt("entrada_treino.csv", entrada, delimiter=',')

In [ ]:
np.savetxt("labels.csv", labels.tolist(), delimiter=',')

In [ ]:
np.savetxt("datasetAtualzizado.csv", df, delimiter = ',')

In [ ]:
x = np.loadtxt("entrada_treino.csv", delimiter=',')
y = np.loadtxt("labels.csv", delimiter=',')

In [ ]:
tamanho_treino = int(x.shape[0]*0.75)

### T = treino
### S = teste

In [ ]:
x_t = x[ :tamanho_treino]
y_t = y[ :tamanho_treino]

x_s = x[tamanho_treino: ]
y_s = y[tamanho_treino: ]

In [ ]:
redeneural = MLPClassifier(
    verbose = True,
    max_iter = 1000,
    tol=0.000001,
    activation='relu',
    learning_rate_init=0.0001
)

In [ ]:
redeneural.fit(x,y)

Iteration 1, loss = 1.62182754
Iteration 2, loss = 1.61586470
Iteration 3, loss = 1.60974256
Iteration 4, loss = 1.60382360
Iteration 5, loss = 1.59797166
Iteration 6, loss = 1.59202737
Iteration 7, loss = 1.58642061
Iteration 8, loss = 1.58075547
Iteration 9, loss = 1.57493676
Iteration 10, loss = 1.56966596
Iteration 11, loss = 1.56395304
Iteration 12, loss = 1.55832836
Iteration 13, loss = 1.55315469
Iteration 14, loss = 1.54781055
Iteration 15, loss = 1.54238151
Iteration 16, loss = 1.53721166
Iteration 17, loss = 1.53191345
Iteration 18, loss = 1.52671082
Iteration 19, loss = 1.52175680
Iteration 20, loss = 1.51667284
Iteration 21, loss = 1.51160095
Iteration 22, loss = 1.50688839
Iteration 23, loss = 1.50176545
Iteration 24, loss = 1.49718394
Iteration 25, loss = 1.49246853
Iteration 26, loss = 1.48779700
Iteration 27, loss = 1.48303295
Iteration 28, loss = 1.47863681
Iteration 29, loss = 1.47401398
Iteration 30, loss = 1.46957853
Iteration 31, loss = 1.46519240
Iteration 32, los

/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(learning_rate_init=0.0001, max_iter=1000, tol=1e-06, verbose=True)

In [ ]:
y_pred = redeneural.predict(x_t)

In [ ]:
accuracy = accuracy_score(y_t, y_pred)

In [ ]:
print("Acurácia: {:.2f}%".format(accuracy * 100))

Acurácia: 82.16%
